In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

### train data

In [2]:
orders = pd.read_csv('../data/driver/driver_order.csv')
history = pd.read_csv('../data/driver/driver_order_products.csv')
history = history.merge(orders, on='order_id', how='inner')
history = history[history['counter'] > 2]
order_pattern = history[history['counter'] <= 12]
order_pattern['counter'] = order_pattern['counter'] - 3
order_pattern = order_pattern[['order_id','user_id','product_id','aisle_id','department_id','counter']]
index = ['user_id','product_id','aisle_id','department_id']
columns = ['counter']
values = ['order_id']
order_pattern = order_pattern.pivot_table(index=index, columns=columns, values=values, aggfunc='count').reset_index()
order_pattern.columns = ["_".join((str(j),str(i))) for i,j in order_pattern.columns]
del history
del orders

In [3]:
orders = pd.read_csv('../data/driver/driver_order.csv')
orders = orders[['order_id','user_id','counter','days_since_prior_order']]
orders = orders[orders['counter'] > 1]
orders['days'] = orders.groupby('user_id')['days_since_prior_order'].shift(1)
orders = orders[orders['counter'] > 2]
orders['days'] = orders.groupby('user_id')['days'].cumsum()
history = pd.read_csv('../data/driver/driver_order_products.csv')
history = history.merge(orders, on='order_id', how='inner')
history = history[['user_id','product_id','days','counter']]
order_days = history.copy()
min_counter = order_days.groupby(['user_id','product_id'])['counter'].min().reset_index()
min_counter = min_counter.rename(columns={'counter':'min_counter'})
order_days = order_days.merge(min_counter, on=['user_id','product_id'])
order_days['counter'] = order_days['counter'] - order_days['min_counter']
order_days = order_days[order_days['counter'] <= 9]
index = ['user_id','product_id']
columns = ['counter']
values = ['days']
order_days = order_days.pivot_table(index=index, columns=columns, values=values).reset_index()
order_days.columns = ["_".join((str(j),str(i))) for i,j in order_days.columns]
del history
del orders

In [4]:
order_pattern = order_pattern.rename(columns={'_user_id':'user_id','_product_id':'product_id'})
order_days = order_days.rename(columns={'_user_id':'user_id','_product_id':'product_id'})

In [5]:
dependent = pd.read_csv('../data/model/dependent/dependent_n_1.csv')

In [6]:
dependent = dependent[['user_id','eval_set','reordered','product_id']]

In [7]:
dependent = dependent.merge(order_pattern, on=['user_id','product_id'])
dependent = dependent.merge(order_days, on=['user_id','product_id'])

In [8]:
dependent.to_csv('../data/catboost/data/train_data1.csv', index=False)

### score data

In [9]:
orders = pd.read_csv('../data/driver/driver_order.csv')
history = pd.read_csv('../data/driver/driver_order_products.csv')
history = history.merge(orders, on='order_id', how='inner')
history = history[history['counter'] > 1]
order_pattern = history[history['counter'] <= 11]
order_pattern['counter'] = order_pattern['counter'] - 2
order_pattern = order_pattern[['order_id','user_id','product_id','aisle_id','department_id','counter']]
index = ['user_id','product_id','aisle_id','department_id']
columns = ['counter']
values = ['order_id']
order_pattern = order_pattern.pivot_table(index=index, columns=columns, values=values, aggfunc='count').reset_index()
order_pattern.columns = ["_".join((str(j),str(i))) for i,j in order_pattern.columns]
del history
del orders

In [10]:
orders = pd.read_csv('../data/driver/driver_order.csv')
orders = orders[['order_id','user_id','counter','days_since_prior_order']]
orders = orders[orders['counter'] > 0]
orders['days'] = orders.groupby('user_id')['days_since_prior_order'].shift(1)
orders = orders[orders['counter'] > 1]
orders['days'] = orders.groupby('user_id')['days'].cumsum()
history = pd.read_csv('../data/driver/driver_order_products.csv')
history = history.merge(orders, on='order_id', how='inner')
history = history[['user_id','product_id','days','counter']]
order_days = history.copy()
min_counter = order_days.groupby(['user_id','product_id'])['counter'].min().reset_index()
min_counter = min_counter.rename(columns={'counter':'min_counter'})
order_days = order_days.merge(min_counter, on=['user_id','product_id'])
order_days['counter'] = order_days['counter'] - order_days['min_counter']
order_days = order_days[order_days['counter'] <= 9]
index = ['user_id','product_id']
columns = ['counter']
values = ['days']
order_days = order_days.pivot_table(index=index, columns=columns, values=values).reset_index()
order_days.columns = ["_".join((str(j),str(i))) for i,j in order_days.columns]
del history
del orders

In [11]:
order_pattern = order_pattern.rename(columns={'_user_id':'user_id','_product_id':'product_id'})
order_days = order_days.rename(columns={'_user_id':'user_id','_product_id':'product_id'})

In [12]:
dependent = pd.read_csv('../data/model/dependent/dependent_n.csv')

In [13]:
dependent = dependent[['user_id','eval_set','reordered','product_id']]

In [14]:
dependent = dependent.merge(order_pattern, on=['user_id','product_id'])
dependent = dependent.merge(order_days, on=['user_id','product_id'])

In [15]:
dependent.to_csv('../data/catboost/data/score_data1.csv', index=False)